In [ ]:
!pip install libauc tabpfn pytorch_tabnet networkx

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import resnet20 as ResNet20
from libauc.datasets import CIFAR10
from libauc.utils import ImbalancedDataGenerator
from libauc.sampler import DualSampler
from libauc.metrics import auc_roc_score
import torch
from PIL import Image
import numpy as np
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score, classification_report, precision_score, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tabpfn import TabPFNClassifier
import networkx as nx

from pytorch_tabnet.tab_model import TabNetClassifier
import torch


# from sklearn.preprocessing import MinMaxScaler

In [ ]:
def set_all_seeds(SEED):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
file_path = '/content/drive/My Drive/Malware_Android_LibAuc/FloodPrediction.csv'
delimiter = ','
flood = pd.read_csv(file_path, delimiter=delimiter)
flood = flood.drop(['Sl'], axis=1)
flood['Flood?'] = flood["Flood?"].fillna(0)
print(flood.shape)

flood.head(5)

In [ ]:
import networkx as nx
graph = nx.Graph()

# Iterate over each data point
for i in range(embedded_docs_shuffled.shape[0]):
    label = trainY_shuffled[i]
    node_id = i
    graph.add_node(node_id)

for i in range(embedded_docs_shuffled.shape[0]):
    for j in range(i + 1, embedded_docs_shuffled.shape[0]):
        if trainY_shuffled[i] == trainY_shuffled[j]:
            # Nodes have the same label, connect them
            node_id1 = i
            node_id2 = j
            graph.add_edge(node_id1, node_id2)
for col in np.array(flood.columns[1:17]):


In [ ]:
flood[flood['Station_Names'] == 0]['Month'].unique()

In [ ]:
flood.isna().sum()

In [ ]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()

In [ ]:
lbl_enc.fit(np.array(flood['Station_Names']))

In [ ]:
flood['Station_Names'] = lbl_enc.transform(np.array(flood['Station_Names']))
flood['Station_Names'].shape

In [ ]:
lbl_enc_st = preprocessing.LabelEncoder()
lbl_enc_st.fit(np.array(flood['Station_Number']))
flood['Station_Number'] = lbl_enc_st.transform(np.array(flood['Station_Number']))
flood['Station_Number']

In [ ]:
flood.columns

In [ ]:
station_names_unique = flood['Station_Names'].unique()
station_names_unique

In [ ]:
flood['ALT'].unique()

In [ ]:
year_unique = flood['Year'].unique()
month_unique = flood['Month'].unique()
max_temp_unique = flood['Max_Temp'].unique()



In [ ]:
X = np.array(flood.drop(['Flood?'], axis=1))
Y = np.array(flood['Flood?'])
X.shape, Y.shape

In [ ]:
X = X.astype(np.float32)
Y = Y.astype(np.int64)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)
X_train.shape, X_val.shape, Y_train.shape, Y_val.shape

In [ ]:
classifier = TabPFNClassifier(device="cpu", N_ensemble_configurations=32)
classifier.fit(X_train[:1024], Y_train[:1024])

In [ ]:
y_eval, p_eval = classifier.predict(X_val, return_winning_probability=True)
accuracy_score(Y_val, y_eval)

In [ ]:
from torchmetrics.classification import BinaryAccuracy
binary_accuracy = BinaryAccuracy()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns


accuracy = accuracy_score(Y_val, y_eval) * 100
print(f'Accuracy: {accuracy}')

precision = precision_score(Y_val, y_eval)
recall = recall_score(Y_val, y_eval)
f1 = f1_score(Y_val, y_eval)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_eval)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
clf_tabnet = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                           scheduler_params={"step_size":5,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                      )

# fit the model
clf_tabnet.fit(
    X_train,Y_train,
    eval_set=[(X_train, Y_train), (X_val, Y_val)],
    eval_name=['train', 'test'],
    eval_metric=['auc','accuracy'],
    max_epochs=500, patience=60,
    batch_size=512, virtual_batch_size=512,
    num_workers=0,
    weights=1,
    drop_last=False
)

In [ ]:
clf_tabnet_ra = TabNetClassifier(optimizer_fn=torch.optim.RAdam,
                       scheduler_params={"step_size":5,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                      )

# fit the model
clf_tabnet_ra.fit(
    X_train,Y_train,
    eval_set=[(X_train, Y_train), (X_val, Y_val)],
    eval_name=['train', 'test'],
    eval_metric=['auc','balanced_accuracy'],
    max_epochs=500, patience=60,
    batch_size=512, virtual_batch_size=512,
    num_workers=0,
    weights=1,
    drop_last=False
)

In [ ]:
y_eval= clf_tabnet.predict(X_val)
print(classification_report(Y_val, y_eval))

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

accuracy = accuracy_score(Y_val, y_eval) * 100
print(f'Accuracy: {accuracy}')

precision = precision_score(Y_val, y_eval)
recall = recall_score(Y_val, y_eval)
f1 = f1_score(Y_val, y_eval)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_eval)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

clf_log = LogisticRegression()
clf_rfc = RandomForestClassifier()
clf_xgb = XGBClassifier()
clf_lgb = lgb.LGBMClassifier()

clf_log.fit(X_train, Y_train)
y_pred_log = clf_log.predict(X_val)

clf_rfc.fit(X_train, Y_train)
y_pred_rfc = clf_rfc.predict(X_val)

clf_xgb.fit(X_train, Y_train)
y_pred_xgb = clf_xgb.predict(X_val)

clf_lgb.fit(X_train, Y_train)
y_pred_lgb = clf_lgb.predict(X_val)

In [ ]:
accuracy_log = (Y_val == y_pred_log).sum()/Y_val.shape[0]
print(f'Accuracy: {accuracy_log}')

precision = precision_score(Y_val, y_pred_log)
recall = recall_score(Y_val, y_pred_log)
f1 = f1_score(Y_val, y_pred_log)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_log)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
accuracy_rfc = accuracy_score(Y_val, y_pred_rfc) * 100
print(f'Accuracy: {accuracy_rfc}')

cr = classification_report(Y_val, y_pred_rfc)
print(cr)

precision = precision_score(Y_val, y_pred_rfc)
recall = recall_score(Y_val, y_pred_rfc)
f1 = f1_score(Y_val, y_pred_rfc)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_rfc)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
accuracy_xgb = accuracy_score(Y_val, y_pred_xgb) * 100
print(f'Accuracy: {accuracy_xgb}')

precision = precision_score(Y_val, y_pred_xgb)
recall = recall_score(Y_val, y_pred_xgb)
f1 = f1_score(Y_val, y_pred_xgb)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_xgb)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
accuracy_lgb = accuracy_score(Y_val, y_pred_lgb) * 100
print(f'Accuracy: {accuracy_lgb}')

precision = precision_score(Y_val, y_pred_lgb)
recall = recall_score(Y_val, y_pred_lgb)
f1 = f1_score(Y_val, y_pred_lgb)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_lgb)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
all_predictions = np.array([y_pred_lgb, y_pred_xgb, y_pred_rfc, y_eval])

# Perform majority voting ensemble by taking the mode (most frequent) prediction for each sample
ensemble_pred = np.round(np.mean(all_predictions, axis=0))

accuracy_ensemble = accuracy_score(Y_val, ensemble_pred) * 100
print(f'Accuracy: {accuracy_ensemble}')

precision = precision_score(Y_val, ensemble_pred)
recall = recall_score(Y_val, ensemble_pred)
f1 = f1_score(Y_val, ensemble_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, ensemble_pred)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(X_train, Y_train)

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=9)
clf_knn.fit(X_train, Y_train)

y_pred_knn = clf_knn.predict(X_val)

accuracy_knn = accuracy_score(Y_val, y_pred_knn) * 100
print(f'Accuracy: {accuracy_knn}')

precision = precision_score(Y_val, y_pred_knn)
recall = recall_score(Y_val, y_pred_knn)
f1 = f1_score(Y_val, y_pred_knn)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_knn)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(X_train, Y_train)

y_pred_gnb = clf_gnb.predict(X_val)

accuracy_gnb = accuracy_score(Y_val, y_pred_gnb) * 100
print(f'Accuracy: {accuracy_gnb}')

precision = precision_score(Y_val, y_pred_gnb)
recall = recall_score(Y_val, y_pred_gnb)
f1 = f1_score(Y_val, y_pred_gnb)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_gnb)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

clf_ada = AdaBoostClassifier(n_estimators=100, random_state=0)
clf_ada.fit(X_train, Y_train)

y_pred_ada = clf_ada.predict(X_val)

accuracy_ada = accuracy_score(Y_val, y_pred_ada) * 100
print(f'Accuracy: {accuracy_ada}')

precision = precision_score(Y_val, y_pred_ada)
recall = recall_score(Y_val, y_pred_ada)
f1 = f1_score(Y_val, y_pred_ada)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_ada)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
clf_mlp = MLPClassifier(random_state=42, max_iter=1000).fit(X_train, Y_train)

In [ ]:
y_pred_mlp = clf_mlp.predict(X_val)

accuracy_mlp = accuracy_score(Y_val, y_pred_mlp) * 100
print(f'Accuracy: {accuracy_mlp}')

precision = precision_score(Y_val, y_pred_mlp)
recall = recall_score(Y_val, y_pred_mlp)
f1 = f1_score(Y_val, y_pred_mlp)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_mlp)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
clf_xtc = ExtraTreesClassifier(n_estimators=100, random_state=42)
clf_xtc.fit(X_train, Y_train)

y_pred_xtc = clf_xtc.predict(X_val)
accuracy_xtc = accuracy_score(Y_val, y_pred_xtc) * 100
print(f'Accuracy: {accuracy_xtc}')

precision = precision_score(Y_val, y_pred_xtc)
recall = recall_score(Y_val, y_pred_xtc)
f1 = f1_score(Y_val, y_pred_xtc)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_xtc)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf_svm.fit(X_train, Y_train)


y_pred_svm = clf_svm.predict(X_val)
accuracy_svm = accuracy_score(Y_val, y_pred_svm) * 100
print(f'Accuracy: {accuracy_svm}')

precision = precision_score(Y_val, y_pred_svm)
recall = recall_score(Y_val, y_pred_svm)
f1 = f1_score(Y_val, y_pred_svm)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


cm = confusion_matrix(Y_val, y_pred_svm)

# Plot confusion matrix using seaborn
plt.figure(figsize=(2, 2))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("True")
# plt.title("Confusion Matrix")
plt.show()